# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [3]:
# Load necessary packages
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *
# remove "object"-type features and SalesPrice from `X`
X = df.drop('SalePrice',axis=1).select_dtypes(exclude = 'object')
# Impute null values
for column in X.columns:
    med = X[column].median()
    X[column].fillna(value = med, inplace = True)
# Create y
y = df.SalePrice

Look at the information of `X` again

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [5]:
# Split in train and test
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.25)

In [8]:
def scoring_params(X_train, y_train, X_test, y_test, kind = 'Linear', alpha = 1):
    fit_method = {'linear':LinearRegression(), 'lasso':Lasso(alpha = alpha),'ridge':Ridge(alpha = alpha)}
    
    reg = fit_method[kind.lower()]
    
    reg.fit(X_train, y_train)
    y_hat= reg.predict(X_train)
    y_hat_test = reg.predict(X_test)
    r2 = reg.score(X_train,y_train)
    r2_test = reg.score(X_test, y_test)
    mse = mean_squared_error(y_train, y_hat)
    mse_test = mean_squared_error(y_test, y_hat_test)

    return (r2, r2_test),(mse,mse_test),reg

In [9]:
# Fit the model and print R2 and MSE for train and test
scoring_params(X_train,y_train,X_test,y_test)

((0.8154153510379787, 0.7595024733628544),
 (1051984102.6186548, 1939689995.7603562),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False))

## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [10]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# Scale the data and perform train test split
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data = X_train_scaled, columns = X_train.columns)
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(data = X_test_scaled, columns = X_train.columns)
X_train_scaled.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,...,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000
mean,0.499371,0.217190,0.166917,0.056052,0.564688,0.574429,0.707543,0.580974,0.073171,0.076971,...,0.333085,0.115358,0.090986,0.054194,0.007710,0.033578,0.003133,0.003886,0.477958,0.456393
std,0.288743,0.248833,0.076551,0.045491,0.153009,0.139666,0.232605,0.343935,0.128941,0.080924,...,0.150756,0.149044,0.129107,0.149980,0.062384,0.125647,0.052100,0.035969,0.241610,0.329610
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.254290,0.000000,0.133562,0.038865,0.444444,0.500000,0.573643,0.283333,0.000000,0.000000,...,0.236953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363636,0.250000
50%,0.500343,0.176471,0.164384,0.051395,0.555556,0.500000,0.720930,0.716667,0.000000,0.065202,...,0.338505,0.000000,0.047809,0.000000,0.000000,0.000000,0.000000,0.000000,0.454545,0.500000
75%,0.746397,0.294118,0.198630,0.064610,0.666667,0.625000,0.930233,0.900000,0.114659,0.125620,...,0.406206,0.204784,0.131474,0.000000,0.000000,0.000000,0.000000,0.000000,0.636364,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Perform the same linear regression on this data and print out R-squared and MSE.

In [11]:
# Your code here
scoring_params(X_train_scaled,y_train,X_test_scaled,y_test)

((0.8154153510379787, 0.789848200210347),
 (1051984102.618655, 1694941937.003953),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False))

## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [12]:
# Create X_cat which contains only the categorical variables
X_cat = df.select_dtypes(include = 'object')

In [13]:
# Make dummies
X_cat = pd.get_dummies(X_cat)
X_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [14]:
# Your code here
X_train_scaled_merged = X_train_scaled.join(X_cat)
X_test_scaled_merged = X_test_scaled.join(X_cat)

Perform the same linear regression on this data and print out R-squared and MSE.

In [15]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged,y_test)

((0.8522138136360442, -7.032985707908106e+19),
 (842262449.8611872, 5.672329445008064e+29),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False))

Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [16]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 1)

((0.8521788935690053, 0.7548399089408757),
 (842461466.1693898, 1977295079.2319436),
 Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False))

With a higher regularization parameter (alpha = 10)

In [17]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 10)

((0.8508320104435955, 0.7697398848179677),
 (850137617.1601908, 1857121975.7909615),
 Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False))

## Ridge

With default parameter (alpha = 1)

In [18]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'ridge', alpha = 1)

((0.8487073667501063, 0.7781099953495694),
 (862246378.1099688, 1789614339.0656512),
 Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001))

With default parameter (alpha = 10)

In [19]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'ridge', alpha = 10)

((0.8216046374337136, 0.7741555151303818),
 (1016710146.0275694, 1821508495.429111),
 Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001))

## Look at the metrics, what are your main conclusions?

Ridge performed the best with the test set at an alpha level of 10 for the data with every predictor. The data with fewer predictors performed better, actually, than everything else.

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [29]:
# number of Ridge params almost zero
_,_,ridge_reg = scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 1)
sum(ridge_reg.coef_<0.00000001)

147

In [30]:
# number of Ridge params almost zero
_,_,ridge_reg = scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 10)
sum(ridge_reg.coef_<0.00000001)

173

In [31]:
# number of Lasso params almost zero
_,_,lasso_reg = scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 10)
sum(lasso_reg.coef_<0.00000001)

173

Compare with the total length of the parameter space and draw conclusions!

In [ ]:
# your code here
len(X_test_scaled_merged.columns)

## Summary

Great! You now know how to perform Lasso and Ridge regression.